# Paso 1: Instalar las librerías y dependencias necesarias

###### El paso inicial, es naturalmente importar todas las librerías y dependencias que estas contienen
###### PyCaret instalará automáticamente Pandas, Numpy, Scikit-learn y Pickle por nosotros como sus dependencias, por lo que no deberemos hacerlo nosotros mismos

In [ ]:
#Installar las librerías necesarias, se debe poner el signo de ! antes del comando, ya que es un comando de shell
#!pip install datasets
#!pip install pycaret

# Paso 2: Descargamos el dataset

###### El segundo paso como está indicado, es descargar el dataset que se encuentra en la página de HuggingFace 

In [ ]:
#Importamos las dependencias a usar
from datasets import load_dataset
import pandas as pd

In [ ]:
#Guardamos dentro de una variable el nombre del dataset a descargar 
dataset = load_dataset('hate_speech_offensive')

In [ ]:
#Convertimos nuestro dataset de entrenamiento en un dataframe de Pandas
df = pd.DataFrame(dataset['train'])

In [ ]:
#Guardamos dentro de un csv nuestro archivo
df.to_csv('hate_speech_offensive.csv', index=False)

# Paso 3: Analizar la estructura de nuestro dataset

###### Un paso muy importante antes de continuar, es analizar cuantos casos tiene por categoría nuestro dataset, de esta forma podremos ver si se trata de un dataset balanceado o no.
###### ¿Es esto importante?, ¿por qué?
###### Bueno cuando estamos trabajando con el entrenamiento de modelos, una de las cosas más importantes es verificar que no existe una sobre representación de ninguna de las categorías, de lo contrario, no tendremos un modelo eficiente, tenderá a tener un mayor nivel de error al predecir categorías si trabajamos con un marco de entrenamiento con sesgos. 

In [ ]:
#Importamos nuestro dataset y lo guardamos en una variable/dataframe llamada df 
df = pd.read_csv('hate_speech_offensive.csv')

In [ ]:
#Mediante pandas contamos la cantidad de tweets que corresponden a cada categoría dentro de la columna class
class_counts = df['class'].value_counts()

In [ ]:
#Esto "imprime" en consola la cantidad de tweets que corresponden a cada categoría
print(class_counts)

# Paso 4: Crear un dataset balanceado

###### Una vez que analizamos nuestro dataset, nos damos cuenta de que el mismo está desbalanceado, lo que debemos hacer entonces es aplicar una técnica mediante un método de Pandas, que es el "muestreo"(sample), para seleccionar solo la cantidad de casos por categoría que nosotros estimemos necesario y le indicaremos que seleccione al azar.
###### Son tres categorías posibles de etiquetas en nuestro dataset, 2: para tweets normales, 1: para tweets de odio(racismo y sexismo) y 0: para tweets con lenguaje ofensivo(insultos generales).
###### Le indicaremos que seleccione todos los tweets con lenguaje ofensivo(etiqueta 0) y de las otras dos categorías que solo seleccione 3mil, ya que consideramos que con eso tendremos un dataset balanceado y listo para entrenar, evitando sesgos.

In [ ]:
#Importamos el dataset en formato csv a usar y lo guardamos dentro de una variable/dataframe llamada "df"
df = pd.read_csv('hate_speech_offensive.csv')

In [ ]:
#Creamos tres dataframes independientes, cada uno conteniendo la cantidad de tweets que le indicamos y por etiqueta, 
# y creando una muestra al azar.
df_0 = df[df['class'] == 0]
df_1 = df[df['class'] == 1].sample(n=3000, random_state=1)
df_2 = df[df['class'] == 2].sample(n=3000, random_state=1)

In [ ]:
#Concatenamos los tres dataframes anteriores para crear uno solo
balanced_df = pd.concat([df_0, df_1, df_2])

In [ ]:
#Guardamos el nuevo dataset balanceado dentro de un nuevo archivo csv, conservando solo las columnas "class" y "tweet"
#Ya que no necesitamos al resto de las columnas para nuestro modelo
balanced_df[['class', 'tweet']].to_csv('fixed_dataset.csv', index=False)

# Paso 5: Limpiar los tweets de caracteres molestos

###### Un paso muy importante, es limpiar los tweets de caracteres que podrían llegar a perjudicar la capacidad analítica de nuestro modelo, esto son los arrobas, enlaces http y otros elementos que no aportan nada a nuestro modelo predictivo, además de convertir todas las palabras a minúsculas para su mejor manejo. Para todo esto, usaremos "expresiones regulares" y usaremos una función lambda(callback) para lograrlo en cada etapa de limpieza.

In [ ]:
#Importamos las dependencias
import re
import numpy as np

In [ ]:
#Importamos el archivo csv generado en el paso anterior que fue el scraping de tweets
df = pd.read_csv('fixed_dataset.csv')

In [ ]:
#Convertimos todos los datos de "tweet" a tipo string para no tener problemas de "tipado"
df['tweet'] = df['tweet'].astype(str)

In [ ]:
#Aplicamos expresiones regulares para eliminar caracteres indeseados, enlaces y arrobamiento, 
#y luego convertimos todo a minúsculas 
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\s+', ' ', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'@\w+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\d+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: x.lower().strip())

In [ ]:
#Reemplazamos los espacios en blanco de la columna Comentario con "Nan"
df['tweet'].replace('', np.nan, inplace=True)

In [ ]:
#Eliminamos todas las filas que tengan valor "Nan" en la celda de la columna "tweet"
df.dropna(subset=['tweet'], inplace=True)

In [ ]:
#Eliminamos todos los tweets duplicados y solo nos quedamos con uno de ellos en caso de existir más de uno
df.drop_duplicates(subset='tweet', keep='first', inplace=True)

In [ ]:
#Aquí seleccionamos solo las columnas "class" y "tweet" para guardar en el nuevo archivo csv
df = df[['class', 'tweet']]

In [ ]:
#Guardamos el resultado de la limpieza sin un índice
df.to_csv('hate_speech_offensive_cleaned.csv', index=False)

# Paso 6: Entrenamos el modelo

###### Luego de que tenemos ya todos los tweets limpios y un dataset balanceado, es momento de la parte más importante de todas, entrenar un modelo de clasificación de tweets.
###### Para ello dividiremos nuestro dataset en dos partes, elegidas cada una de ellas al azar(esto es importantísimo), un 15% será destinada a probar el modelo y el 85% restante está destinado a entrenar dicho modelo.
###### 

In [ ]:
#Importamos las librerías y dependencias
from pycaret.classification import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
#Cargar los datos que vayamos a usar
df = pd.read_csv('hate_speech_offensive_cleaned.csv', sep=',')

In [ ]:
#Nos aseguramos de que las columnas tengan el nombre correcto para que PyCaret haga un trabajo adecuado, por ello le cambiaremos
#el nombre a las columnas "class" pasará a llamarse "Label" y "tweet" pasará a llamarse "Text"
df.columns = ['Label', 'Text']

In [ ]:
#Transformamos los textos en características utilizando TF-IDF. Este proceso asigna un puntaje a cada palabra en un tweet
#dependiendo de cuántas veces aparece en el tweet y cuántas veces aparece en el resto del conjunto de datos. 
#Esto nos ayuda a destacar las palabras que son particularmente importantes en un tweet.
#Usaremos la opción de quitar las palabras de parada ('stop-words'), ya que estas palabras (como 'a', 'and', 'the' en inglés) 
#aparecen tan frecuentemente que no aportan mucha información al modelo.
#El parámetro max_features limita el número de palabras que consideraremos. En este caso, solo estamos tomando 
#las 1000 palabras más frecuentes.
#El resultado de este proceso es una matriz donde cada fila representa un tweet y cada columna representa una de las 
#1000 palabras más frecuentes. El valor en una celda específica es el puntaje TF-IDF de una palabra para un tweet en particular.
#En resumen, esta matriz nos ayuda a entender qué palabras son especialmente características para cada tweet y
#nos permite predecir a qué categoría pertenece un tweet basándonos en estas palabras.

vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
features = vectorizer.fit_transform(df['Text'])

In [ ]:
#Creamos un nuevo DataFrame con las características TF-IDF y las guardamos dentro de arrays
df_tfidf = pd.DataFrame(features.toarray(), columns=vectorizer.get_feature_names_out())
df_tfidf['Label'] = df['Label']

In [ ]:
#Dividir los datos en conjunto de entrenamiento y prueba
train_df, test_df = train_test_split(df_tfidf, test_size=0.15, random_state=21)

In [ ]:
#Guardamos el vectorizador para usarlo en la etapa de predicción
pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))

In [ ]:
#Configuramos el módulo de clasificación de PyCaret
clf = setup(data = train_df, target = 'Label', session_id = 21, n_jobs = -1)

In [ ]:
#Comparamos los modelos para ver cuál tiene mayor capacidad de predicción
best = compare_models()

In [ ]:
#Pasamos a seleccionar el modelo que mejor resultado tuvo y lo finalizamos
final_model = finalize_model(best)

In [ ]:
#Guardamos el modelo que en la carpeta de destino
save_model(final_model, model_name = "hate_tweets_model")

# Paso 7: Probamos nuestro modelo

###### El último paso de todos es probar si nuestro modelo funciona con una serie de "tweets" que crearemos para ver que tan efectivo resultó el entrenamiento realizado y si puede predecir efectivamente a la categoría que pertenece cada uno.
###### Esto es sumamente importante antes de poner al modelo en "producción" (usar el modelo para nuestro propósito), ya que sino, no sabremos si funciona adecuadamente y pondría en riesgo nuestro trabajo.

In [ ]:
#Importar las librerías y dependencias
from pycaret.classification import load_model, predict_model
import pickle

In [ ]:
#Cargamos el modelo
modelo = load_model('hate_tweets_model')

In [ ]:
#Cargamos el vectorizador
vectorizer = pickle.load(open("tfidf_vectorizer.pkl", 'rb'))

In [ ]:
#Creamos un nuevo tweet para probar si el modelo funciona
nuevo_tweet = pd.DataFrame({"Text": ["You are such a stupid bitch, damned fool"]})

In [ ]:
#Transformamos el tweet de prueba al mismo formato que los datos de entrenamiento
tweet_transformado = vectorizer.transform(nuevo_tweet['Text'])
df_tfidf = pd.DataFrame(tweet_transformado.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
#Predecimos la categoría a la que pertenece el tweet de prueba
prediccion = predict_model(modelo, data=df_tfidf)

In [ ]:
#Imprimimos todo el DataFrame de predicciones, para ver si ha hecho una buena predicción de nuestro nuevo tweet
print(prediccion)